In [34]:
from pprint import pprint
from proxy import Resolve

r = Resolve('/Volumes/thom_7_for_Mac/tmp/素材')

r.project.GetSetting()
# r.project.SetSetting('colorScienceMode', 'davinciYRGBColorManagedv2')
# r.project.SetSetting('colorSpaceInput', 'S-Gamut3.Cine/S-Log3')
# r.project.SetSetting('colorAcesIDT', 'Sony SLog3 SGamut3Cine')
# r.project.SetSetting('colorAcesIDT', 'Panasonic V35')

# r.timeline.GetSetting()

# for subfolder in r.root_folder.GetSubFolderList():
#     for clip in subfolder.GetClipList():
#         pprint(clip.GetClipProperty())
#         break

for subfolder in r.root_folder.GetSubFolderList():
    if subfolder.GetClipList():
        print("---")

---
